<a href="https://colab.research.google.com/github/eastmountaincode/DSC/blob/main/howToGetReviewDataFromGoogleMaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following is a documentation of the process of getting review data from Google Maps using a combination of the Google Places API and APIFY, a web scraping service (https://apify.com/). We use the Google Places API to get a list of businesses, and then we use APIFY to scrape the reviews.

In [ ]:
#Importing libraries
import pandas as pd
import requests, json
import time

You will need an API key for the Google Places API. This can be obtained by making a Google Cloud account. You can learn how to obtain and API key at this link: https://developers.google.com/maps/documentation/places/web-service/get-api-key

In [ ]:
api_key = 'api key goes here'

In [ ]:
#Our base URL that we will attach parameters to
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

## If getting data from just one location

In [ ]:
# This is the text string on which to search. This should be altered depending on what you are looking for. 
# For example, you might set query as "Honky Tonk in Nashville, TN" or "Coffee shop in San Francisco".

query = "Urgent Care in Cincinnati"
  
#The query and API key are attached to the base URL
r = requests.get(url + 'query=' + query +
                        '&key=' + api_key)
  
#we convert the response from requestions into a json object
currentJSON = r.json()


#This is the dataframe where we'll keep all the info we get from the json(s). This initializes an empty dataframe with some defined columns.
holdingDF = pd.DataFrame(columns=["bizName", "placeID", "address", "placeTypes", "numOfRatings"])

#each json we get from our get request will have 50 businesses. If there are more businesses, there will be a 'next_page_token'.
#we want the following while loop to run at least once regardless of what we get in our json, so we set isToken to True.
isToken = True

while isToken:
  for business in range(len(currentJSON["results"])):
    if 'user_ratings_total' in currentJSON["results"][business]:
      bizName = currentJSON["results"][business]['name']
      placeID = currentJSON["results"][business]['place_id']
      address = currentJSON["results"][business]['formatted_address']

      allTypes = ""
      for eachType in currentJSON["results"][business]['types']:
        allTypes = allTypes + ", " + eachType
      allTypes = allTypes[1:]
      
      numOfRatings = currentJSON["results"][business]['user_ratings_total']

      new_row = {'bizName': bizName,
                'placeID': placeID,
                'address': address,
                'placeTypes': allTypes,
                'numOfRatings': numOfRatings}
      holdingDF = holdingDF.append(new_row, ignore_index=True)

  time.sleep(1.5)
  #The while loop checks if there is a next_page_token after the loop is complete, but we need this here to check if the FIRST page has one
  #If it does, go ahead and load the next JSON. Do display(currentJSON) if you want to get a better idea of how this works.
  if 'next_page_token' in currentJSON:
    r = requests.get(url + '&key=' + api_key + '&pagetoken=' + currentJSON['next_page_token'])
    
  currentJSON = r.json()
  if 'next_page_token' in currentJSON:
    isToken = True
  else:
    isToken = False


In [ ]:
display(holdingDF)

## If getting data from many locations

Maybe we don't want to perform just one search, maybe we want to perform several searches to get results from many different places. For example, what if we wanted to do search in every city in Ohio? This code can help us do that.

For this code, I used a CSV file containting every city in Ohio ordered by population. I got this list from this link: https://www.ohio-demographics.com/cities_by_population (it required a bit of cleaning).

In [ ]:
#Importing location data
from google.colab import drive
drive.mount('/content/drive')
cityDF = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ERURproject/Yelp/OhioPopulationByCity.csv", index_col= 0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
holdingDF = pd.DataFrame(columns=["bizName", "placeID", "address", "placeTypes", "numOfRatings"])
counter = 0

#I limited this search to the first 10 most populated cities in Ohio, hence the range(10).
for city in range(10):

  #The city name gets passed into the query based on the iteration index
  query = "Urgent Care in {}, Ohio".format(cityDF.loc[city, 'City'])

  r = requests.get(url + 'query=' + query +
                          '&key=' + api_key)
  currentJSON = r.json()
  isToken = True
  while isToken:
    for business in range(len(currentJSON["results"])):
      if 'user_ratings_total' in currentJSON["results"][business]:
        bizName = currentJSON["results"][business]['name']
        placeID = currentJSON["results"][business]['place_id']
        address = currentJSON["results"][business]['formatted_address']

        allTypes = ""
        for eachType in currentJSON["results"][business]['types']:
          allTypes = allTypes + ", " + eachType
        allTypes = allTypes[1:]
        
        numOfRatings = currentJSON["results"][business]['user_ratings_total']

        new_row = {'bizName': bizName,
                  'placeID': placeID,
                  'address': address,
                  'placeTypes': allTypes,
                  'numOfRatings': numOfRatings}
        holdingDF = holdingDF.append(new_row, ignore_index=True)

    
    time.sleep(1.5)
    if 'next_page_token' in currentJSON:
      r = requests.get(url + '&key=' + api_key + '&pagetoken=' + currentJSON['next_page_token'])
      
    # json method of response object convert
    #  json format data into python format data
    currentJSON = r.json()
    if 'next_page_token' in currentJSON:
      isToken = True
    else:
      isToken = False
  counter += 1
  print(str(round((counter/10)*100, 1)) + " percent complete")

#Since we're iterating over many cities, some of our search results may overlap. It's important to remove duplicates.
holdingDF = pd.DataFrame.drop_duplicates(holdingDF)
holdingDF.reset_index(drop=True, inplace=True)

In [ ]:
display(holdingDF)

## Now that we have a dataframe full of businesses from the Google Places API, the next step is to create a list of URLs we can feed to APIFY.

In [ ]:
#Create a new empty dataframe
businessDFurlsDF = pd.DataFrame(columns=["name", "validURL", "placeID", "address", "city", "state", "dataSource"])

In [ ]:
#For the placeID of every business...
for placeIDindex in range(len(holdingDF)):
  
  url = "https://maps.googleapis.com/maps/api/place/details/json?"
  placeIDiter = holdingDF.loc[placeIDindex, 'placeID']  

  #Notice that we're now feeding requests a 'place_id=' parameter now
  r = requests.get(url + 'place_id=' + placeIDiter +
                          '&fields=' + "name,rating,url,formatted_address" + '&key=' + api_key)
  currentJSON = r.json()

  bizName = currentJSON['result']['name']

  #This is the important part; the url in the json response for the business gotten by the placeID
  #The url contains a 'CID number'. When we append this CID number to the URL form below
  #It takes the form of a URL that APIFY can read. 
  placeIDtoSave = currentJSON['result']['url']
  theGoldenCID = placeIDtoSave.split('cid=')[1]
  validURL = 'https://www.google.com/maps/place/?cid=' + theGoldenCID

  address = currentJSON['result']['formatted_address']

  city = address.split(',')[1].strip()
  state = address.split(',')[2].strip()[0:2]

  #We've gotten some extra information like address ad city, but the important colimn is 'validURL'. 
  #This is what we'll use in the next step
  new_row = {'name': bizName,
            'validURL': validURL,
            'placeID': placeIDiter,
            'address': address,
            'city': city,
            'state': state,
              'dataSource': "GoogleMaps"}
  businessDFurlsDF = businessDFurlsDF.append(new_row, ignore_index=True)



In [ ]:
validPlaceIDlist = []

for validURL in range(len(businessDFurlsDF)):
  validPlaceIDlist.append(businessDFurlsDF.loc[validURL, 'validURL'])

#remove duplicates
newlist = list(set(validPlaceIDlist))

In [ ]:
display(newlist)

## Now download the list

In [ ]:
from google.colab import files
df = pd.DataFrame(newlist)
df.to_csv('placeIDlistforAPIFY.csv', index=False)
files.download('placeIDlistforAPIFY.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Now we go to APIFY. Create a new "task" and choose the Google Maps Scraper. Plug in the CSV list that we made in the last step to "Start URLs" (choose upload text file). Remove search terms. Set max crawled places to 0. Under output configuration set max reviews to 99999. Set max images to 0. Uncheck include popular times. Sort reviews by "Newest". Run the scraper. APIFY will give you a JSON file. Download it and bring it into this code. 

Using the previous code, I ran a search for "Urgent Care in Cincinnati", received 40 businesses, and put the CSV file in APIFY. I got back a JSON, which I'm importing into the code now.

In [ ]:
#Mount the drive in case we haven't already.
from google.colab import drive
drive.mount('/content/drive')

dataFromAPIFYDF = pd.read_json("/content/drive/My Drive/Colab Notebooks/Google/GoogleTutorialData.json")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## If you want to make a dataframe where each row is a separate review...

In [ ]:
#REVIEW SCALE


reviewScaleDF = pd.DataFrame(columns=["businessTitle", "googleURL", "CIDnum", "phoneNum", "address", "city", "state", "businessRating", "reviewDate", "reviewText", "reviewRating", "bizType", "dataSource"])
counter = 0
for business in range(len(dataFromAPIFYDF)):
  #Our search will have given us businesses that are not urgent care centers, and since Cincinnati is very close to Kentucky, we may get some results from KY
  #This next step filters results to only include businesses that are in Ohio and that have the categoryName 'Urgent care center'
  if dataFromAPIFYDF.loc[business, 'categoryName'] == 'Urgent care center' and 'OH' in dataFromAPIFYDF.loc[business, 'address']:
    for review in range(len(dataFromAPIFYDF.loc[business, 'reviews'])):

      businessTitle = dataFromAPIFYDF.loc[business, 'title']
      directURL = dataFromAPIFYDF.loc[business, 'placeId']
      placeIDtoSave = dataFromAPIFYDF.loc[business, 'placeId']

      theGoldenCID = placeIDtoSave.split('cid=')[1]

      phoneNum = dataFromAPIFYDF.loc[business, 'phone']
      address = dataFromAPIFYDF.loc[business, 'address']
      city = address.split(',')[1].strip()
      state = address.split(',')[2].strip()[0:2]

      overallRating = dataFromAPIFYDF.loc[business, 'totalScore']
      theDate = dataFromAPIFYDF.loc[business, 'reviews'][review]['publishedAtDate'][0:10]

      reviewText = dataFromAPIFYDF.loc[business, 'reviews'][review]['text']
      reviewRating = dataFromAPIFYDF.loc[business, 'reviews'][review]['stars']


      if reviewText:
        new_row = {'businessTitle': businessTitle,
                  'googleURL': directURL,
                  'CIDnum': theGoldenCID,
                  'phoneNum': phoneNum,
                  'address': address,
                  'city': city,
                  'state': state,
                  'businessRating': overallRating,
                  'reviewDate': theDate,
                  "reviewText": reviewText,
                  "reviewRating": reviewRating,
                  "bizType": "urgentCare",
                  "dataSource": "Google"}
        reviewScaleDF = reviewScaleDF.append(new_row, ignore_index=True)
  counter += 1
  number = (counter/len(dataFromAPIFYDF)*100)
  print(round(number, 2))


In [ ]:
display(reviewScaleDF)

In [ ]:
from google.colab import files
reviewScaleDF.to_csv("reviewScaleUrgentCareGoogle.csv")
files.download("reviewScaleUrgentCareGoogle.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## If you want to make a dataframe where each row is one business, and the review text is stored in a column with one big review, a concatenation of ALL that reviews for that business...

In [ ]:
#INSTITUTION SCALE


institutionalScaleDF = pd.DataFrame(columns=["businessTitle", "googleURL", "CIDnum", "phoneNum", "address", "city", "state", "businessRating", "bigText", "numberOfReviews", "bizType", "dataSource"])
counter = 0
for business in range(len(dataFromAPIFYDF)):
  #Our search will have given us businesses that are not urgent care centers, and since Cincinnati is very close to Kentucky, we may get some results from KY
  #This next step filters results to only include businesses that are in Ohio and that have the categoryName 'Urgent care center'
  if dataFromAPIFYDF.loc[business, 'categoryName'] == 'Urgent care center' and 'OH' in dataFromAPIFYDF.loc[business, 'address']:

    businessTitle = dataFromAPIFYDF.loc[business, 'title']
    directURL = dataFromAPIFYDF.loc[business, 'placeId']
    placeIDtoSave = dataFromAPIFYDF.loc[business, 'placeId']

    theGoldenCID = placeIDtoSave.split('cid=')[1]

    phoneNum = dataFromAPIFYDF.loc[business, 'phone']
    address = dataFromAPIFYDF.loc[business, 'address']
    city = address.split(',')[1].strip()
    state = address.split(',')[2].strip()[0:2]

    overallRating = dataFromAPIFYDF.loc[business, 'totalScore']

    bigText = ""
    numOfReviews = 0
    #For each review...
    for review in range(len(dataFromAPIFYDF.loc[business, 'reviews'])):
      #If the review has text, meaning it's not just a rating based on number of stars...
      if dataFromAPIFYDF.loc[business, 'reviews'][review]['text']:
        reviewText = dataFromAPIFYDF.loc[business, 'reviews'][review]['text']
        #...append the review to this string called big text, and separate individual reviews with a newline, 10 tildes, and another newline.
        bigText = bigText + reviewText
        bigText = bigText + "\n" + "~~~~~~~~~~" + "\n"
        numOfReviews += 1
        


    #Make sure the business had a least ONE review with text in it...
    if bigText:
      new_row = {'businessTitle': businessTitle,
                'googleURL': directURL,
                'CIDnum': theGoldenCID,
                'phoneNum': phoneNum,
                'address': address,
                'city': city,
                'state': state,
                'businessRating': overallRating,
                "bigText": bigText,
                "numberOfReviews": numOfReviews,
                "bizType": "urgentCare",
                "dataSource": "Google"}
      institutionalScaleDF = institutionalScaleDF.append(new_row, ignore_index=True)
  counter += 1
  number = (counter/len(dataFromAPIFYDF)*100)
  print(round(number, 2))



In [ ]:
display(institutionalScaleDF)

In [ ]:
from google.colab import files
institutionalScaleDF.to_csv("institutionalScaleUrgentCareGoogle.csv")
files.download("institutionalScaleUrgentCareGoogle.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>